# ライン情報と選手の脚質を取得する

## 取得元: WinTicket

In [57]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [58]:
# racecard_url = "https://www.winticket.jp/keirin/keiokaku/racecard/2021010127/2/3"
# raceresult_url = "https://www.winticket.jp/keirin/yokkaichi/raceresult/2022010448/3/8"
# req = requests.get(raceresult_url)
# soup = BeautifulSoup(req.content, 'html.parser')

### 日程を与えるとその日開催されたレースのURLをリストで返す関数

In [59]:
def get_urls(year, month, day):
    url = "https://www.winticket.jp/keirin/racecard/" + str(year) + str(month).zfill(2) + str(day).zfill(2)
    req = requests.get(url)
    soup = BeautifulSoup(req.content, 'html.parser')
    url_soup = soup.find_all('a', rel='nofollow')
    urls = [i.get('href') for i in url_soup]
    return urls

 ### urlを与えると脚質のデータフレームを返す関数

In [60]:
def create_legtype_table(url):
    race_tables = pd.read_html(url)
    legtype_table = race_tables[1][['車', '脚']].rename(columns={'車':'no', '脚':'leg'}).droplevel(1, axis=1)
    return legtype_table

### htmlを与えるとライン情報のデータフレームを返す関数

In [61]:
def create_line_table(html, number_of_people):
    line_soup = html.find_all("div", class_="sc-1y958x7-0")
    line_text = [i.text for i in line_soup][0]
    if '競' in line_text:
        return None
    lines = line_text.split('区切り')

    line_data = [[0 for j in range(number_of_people)] for i in range(3)]
    for n, i in enumerate(lines):
        people = len(i)
        for k, j in enumerate(i):
            index = int(j) - 1
            line_data[0][index] = people
            line_data[1][index] = n
            line_data[2][index] = k
    line_data.insert(0, [i+1 for i in range(number_of_people)])
    df = pd.DataFrame({'no':line_data[0],
                       'people':line_data[1],
                       'group':line_data[2],
                       'number':line_data[3]})
    return df

### urlを与えると着順のデータフレームを返す関数

In [62]:
def create_result_table(url):
    result_tables = pd.read_html(url)
    result_table = result_tables[0][['着','車']].rename(columns={'着':'no', '車':'result'})
    return result_table

## ここからメイン

In [63]:
import time
import traceback
from tqdm import tqdm

1. レース情報が乗っているページのURLを取得

In [64]:
race_urls = []
for year in range(2021, 2022):
    for month in range(1, 2):
        for day in tqdm(range(1, 2)):
            race_urls.extend(get_urls(year, month, day))
            time.sleep(1)

100%|██████████| 1/1 [00:01<00:00,  1.42s/it]


2. URLにアクセスして各種データを取得

In [67]:
race_data_7 = pd.DataFrame()
race_data_9 = pd.DataFrame()

for url in tqdm(race_urls):
    #レースIDの設定
    split_url = url.split('/')
    race_id = split_url[4] + "/" + split_url[5] + "/" + split_url[6]

    if (race_id in race_data_7.index) or (race_id in race_data_9.index):
        continue

    try:
        racecard_url = "https://www.winticket.jp" + url
        legtype_table = create_legtype_table(racecard_url)
        if legtype_table.empty or ('欠' in legtype_table['no']):
            continue
        time.sleep(1)

        number_of_people = len(legtype_table)

        req = requests.get(racecard_url)
        soup = BeautifulSoup(req.content, 'html.parser')
        line_table = create_line_table(soup, number_of_people)
        time.sleep(1)

        raceresult_url = racecard_url.replace("racecard", "raceresult")
        result_table = create_result_table(raceresult_url)
        time.sleep(1)

        #dfの作成，結合
        df = pd.concat([legtype_table, line_table, result_table], axis=1)
        df = df.loc[:,~df.columns.duplicated()]

        df.index = [race_id for i in range(len(df))]

        if number_of_people == 7:
            race_data_7 = pd.concat([race_data_7, df])
        elif number_of_people == 9:
            race_data_9 = pd.concat([race_data_9, df])
        else:
            continue
    except:
        print(url)
        print(traceback.format_exc())
        break

100%|██████████| 85/85 [04:59<00:00,  3.53s/it]


## データの前処理

In [87]:
data_7 = race_data_7
data_9 = race_data_9

In [88]:
leg_data = pd.get_dummies(data_7['leg'], columns=['逃', '両', '追'])
data_7 = pd.concat([data_7, leg_data], axis=1).drop('leg', axis=1)
data_7_result = data_7['result']
data_7 = data_7.drop('result', axis=1)
data_7

,no,people,group,number,両,追,逃
2020123045/3/3,1,1,1,0,0,1,0
2020123045/3/3,2,2,0,0,0,1,0
2020123045/3/3,3,2,2,1,0,1,0
2020123045/3/3,4,2,3,1,0,1,0
2020123045/3/3,5,2,2,0,1,0,0
...,...,...,...,...,...,...,...
2021010181/1/9,3,2,1,1,0,1,0
2021010181/1/9,4,2,1,0,0,0,1
2021010181/1/9,5,2,0,0,0,0,1
2021010181/1/9,6,2,2,0,0,0,1


In [89]:
data_7[['両', '追', '逃']] = data_7[['両', '追', '逃']].astype('int64')

In [92]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

In [93]:
model = KMeans(n_clusters=7)
pred = model.fit_predict(data_7)